# CRIANDO O MODELO PARA RECONHECIMENTO DE IMAGEM 


### O Desafio proposto era criar um modelo de reconhecimento de imagem, onde pudesse reconhecer quais imagens eram de gatos e quais eram as de cachorros.

> **Carregando as bibliotecas**

> Foram utilizadas as bibliotecas: tensorflow, keras, ImageDataGenerator, VGG16, Model, Dense e GlobalAveragePooling2D do módulo tensorflow.keras.

In [1]:
#Carregando as bibliotecas
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

> Definimos as configurações para o treinamento do modelo, incluindo a largura e altura das imagens (img_width e img_height), o tamanho do lote (batch_size), os diretórios de treinamento e validação (train_data_dir e validation_data_dir) e o número de épocas (epochs).

In [2]:
# Configurações
img_width, img_height = 150, 150
batch_size = 16
train_data_dir = r'C:\Users\Nayar\OneDrive\Documentos\Nayara Valevskii\Infinity School\Desafio Redes Neurais\img_treino'
validation_data_dir = r'C:\Users\Nayar\OneDrive\Documentos\Nayara Valevskii\Infinity School\Desafio Redes Neurais\img_validacao'
epochs = 10

> O VGG16 é um modelo de rede neural convolucional (CNN) muito conhecido e eficaz, amplamente utilizado para tarefas de classificação de imagens. Nesta linha de código, estamos criando uma instância do modelo VGG16 como base para o nosso modelo de classificação de cachorros e gatos.

> Weights='imagenet': Especificamos o conjunto de pesos a serem usados pelo modelo. O valor 'imagenet' indica que queremos carregar os pesos pré-treinados do modelo VGG16 treinado no conjunto de dados ImageNet. Esses pesos foram treinados em um conjunto enorme e diversificado de imagens e podem capturar características gerais úteis para tarefas de classificação de imagens.

> include_top=False: Isso indica que não queremos incluir as camadas finais de classificação do VGG16, que são responsáveis por mapear as características aprendidas para as classes do conjunto de dados ImageNet original. Ao definir include_top=False, excluímos essas camadas finais e teremos a flexibilidade de adicionar nossas próprias camadas personalizadas para nossa tarefa de classificação de cachorros e gatos.

> input_shape=(img_width, img_height, 3): Especificamos a forma das imagens de entrada que o modelo VGG16 espera receber. As dimensões (img_width, img_height) indicam o tamanho desejado para as imagens de entrada (150x150 pixels neste caso), e 3 representa os três canais de cor (vermelho, verde e azul - RGB). É importante fornecer imagens de entrada com a mesma forma definida aqui para garantir que o modelo funcione corretamente.

In [3]:
# Carregar o modelo pré-treinado (VGG16) com os pesos do ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

58889256/58889256 [==============================] - 3s 0us/step


> Congelamos as camadas do modelo pré-treinado definindo layer.trainable = False para cada camada no modelo.

In [4]:
# Congelar as camadas do modelo pré-treinado
for layer in base_model.layers:
    layer.trainable = False

> Adicionamos novas camadas no topo do modelo. Primeiro, aplicamos um GlobalAveragePooling2D para reduzir as dimensões da saída. Em seguida, adicionamos uma camada Dense com 512 unidades e função de ativação ReLU. Por fim, adicionamos uma camada de saída Dense com 2 unidades (uma para cada classe - cachorro e gato) e função de ativação softmax para obter as probabilidades de cada classe.

In [5]:
# Adicionar novas camadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

> Criamos o modelo final, especificando as entradas (inputs=base_model.input) e as saídas (outputs=predictions).

In [7]:
# Criar o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

> Compilamos o modelo usando o otimizador 'adam', a função de perda 'categorical_crossentropy' e a métrica 'accuracy' para avaliar o desempenho do modelo durante o treinamento.

In [8]:
# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


> Criamos geradores de dados para o treinamento e a validação usando ImageDataGenerator. Esses geradores aplicam várias transformações nas imagens, como redimensionamento, rotação, zoom e espelhamento horizontal, para enriquecer o conjunto de dados de treinamento e evitar overfitting.

> O gerador de treinamento train_generator é criado a partir do diretório de treinamento, especificando o tamanho das imagens alvo, o tamanho do lote e o modo 'categorical' para tratar as classes como variáveis categóricas.

> O gerador de validação validation_generator é criado de forma semelhante, mas a partir do diretório de validação.

In [9]:
# Criar geradores de dados para treinamento e validação
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height),
                                                        batch_size=batch_size, class_mode='categorical')

Found 202 images belonging to 2 classes.
Found 207 images belonging to 2 classes.


> Realizamos o ajuste fino do modelo chamando model.fit(), passando o gerador de treinamento, o número de épocas e o gerador de validação. Durante o treinamento, o modelo será ajustado para melhor se adequar ao conjunto de dados.

In [10]:
# Realizar o ajuste fino do modelo
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
13/13 [==============================] - 212s 17s/step - loss: 0.7534 - accuracy: 0.5495 - val_loss: 0.5062 - val_accuracy: 0.8213
Epoch 2/10
13/13 [==============================] - 300s 24s/step - loss: 0.4936 - accuracy: 0.7772 - val_loss: 0.3984 - val_accuracy: 0.8454
Epoch 3/10
13/13 [==============================] - 298s 24s/step - loss: 0.4068 - accuracy: 0.8267 - val_loss: 0.3562 - val_accuracy: 0.8261
Epoch 4/10
13/13 [==============================] - 348s 28s/step - loss: 0.3396 - accuracy: 0.8614 - val_loss: 0.3527 - val_accuracy: 0.8357
Epoch 5/10
13/13 [==============================] - 334s 27s/step - loss: 0.3016 - accuracy: 0.8713 - val_loss: 0.2570 - val_accuracy: 0.9034
Epoch 6/10
13/13 [==============================] - 198s 15s/step - loss: 0.2810 - accuracy: 0.8762 - val_loss: 0.2870 - val_accuracy: 0.8502
Epoch 7/10
13/13 [==============================] - 98s 8s/step - loss: 0.2947 - accuracy: 0.8861 - val_loss: 0.3042 - val_accuracy: 0.8502
Epoch 8/

> Após o treinamento, salvamos o modelo treinado em um arquivo chamado 'cachorro_gato_modelo_treinado.h5' usando model.save().

In [11]:
# Salvar o modelo treinado
model.save('cachorro_gato_modelo_treinado.h5')

c:\Users\Nayar\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


> **Esse código cria e treina um modelo baseado no VGG16 com camadas personalizadas para classificação de imagens de cachorro e gato. O ajuste fino é realizado para adaptar o modelo pré-treinado ao seu problema específico. Os geradores de dados são usados para fornecer lotes de imagens durante o treinamento, aplicando transformações para melhorar a diversidade dos dados.**

## **TESTANDO O MODELO CRIADO**

> Primeiro, importamos as bibliotecas necessárias: tensorflow, keras, load_img e img_to_array do módulo tensorflow.keras.preprocessing.image, e numpy.

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

> Carregamos o modelo treinado usando keras.models.load_model().

In [13]:
# Carregar o modelo treinado
model = keras.models.load_model('cachorro_gato_modelo_treinado.h5')  # Certifique-se de ter o arquivo do modelo treinado

> Mapeamos os rótulos das classes para interpretar os resultados da previsão. Nesse caso, usamos o dicionário class_labels, onde o índice 0 corresponde a "Cachorro" e o índice 1 corresponde a "Gato".

In [14]:
# Mapear os rótulos das classes
class_labels = {0: 'Cachorro', 1: 'Gato'}

> Definimos a função fazer_previsao(imagem_path) para fazer a previsão de uma única imagem. Ela recebe o caminho da imagem como argumento.

> Dentro da função, carregamos a imagem usando load_img, redimensionamos a imagem para o tamanho desejado (150x150 pixels neste caso), convertemos a imagem em um array com img_to_array, expandimos as dimensões do array para que fique com o formato correto (1, 150, 150, 3) e normalizamos os valores dos pixels dividindo por 255.0.

> Em seguida, usamos o modelo treinado para fazer a previsão chamando model.predict(imagem). Isso retorna uma matriz de previsões para cada classe, onde cada valor representa a probabilidade da imagem pertencer a essa classe.

> Usamos np.argmax(previsao) para obter o índice da classe com a maior probabilidade, ou seja, a classe prevista.

> Com base no índice da classe prevista, mapeamos o rótulo correspondente usando o dicionário class_labels.

> Retornamos o rótulo predito.

In [18]:
# Função para fazer previsão de uma única imagem
def fazer_previsao(imagem_path):
    imagem = load_img(imagem_path, target_size=(150, 150))  # Redimensionar a imagem conforme necessário
    imagem = img_to_array(imagem)
    imagem = np.expand_dims(imagem, axis=0)
    imagem = imagem / 255.0  # Normalizar os valores dos pixels

    # Fazer a previsão com o modelo
    previsao = model.predict(imagem)
    classe_predita = np.argmax(previsao)
    rotulo_predito = class_labels[classe_predita]

    return rotulo_predito

> Em seguida, definimos duas variáveis imagem1 e imagem2 com os caminhos para as imagens de teste.

> Chamamos a função fazer_previsao() para fazer a previsão de cada imagem.

> Por fim, imprimimos os resultados das previsões no console.

In [31]:
# Testar a previsão com algumas imagens de exemplo
imagem1 = r'C:\Users\Nayar\OneDrive\Documentos\Nayara Valevskii\Infinity School\Desafio Redes Neurais\cachorro4.jpg'
imagem2 = r'C:\Users\Nayar\OneDrive\Documentos\Nayara Valevskii\Infinity School\Desafio Redes Neurais\cachorro5.jpg'

previsao1 = fazer_previsao(imagem1)
previsao2 = fazer_previsao(imagem2)

print(f'A imagem1 é um(a) {previsao1}')
print(f'A imagem2 é um(a) {previsao2}')

1/1 [==============================] - 0s 145ms/step
A imagem1 é um(a) Cachorro
A imagem2 é um(a) Cachorro


**Observações Finais** : O modelo criado teve um accuracy bem positiva, mas ele ainda não deve ser usado como o modelo ideal para reconhecimento dessas imagens. O motivo é que ele foi treinado em cima de uma base de dados pequena, e com certeza ao passar outras imagens pelo modelo, pode não reconhecer corretamente, pela falta de treinamento efetivo.

O intuito do desafio era apenas melhorar o código dado anteriormente pelo professor, então foram feitas modificações na base de dados para o treinamento, incluindo mais imagens na internet e mudando a biblioteca de Perceptron para Tensorflow.